# Extract 10-K filings from 2015 to present

In [3]:
# S&P 500 Management Discussion & Analysis Extractor
# This script extracts Item 7 (MD&A) text from 10-K filings from 2015 to present
# of S&P 500 companies and exports it to a CSV file.

import pandas as pd
import time
from edgar import *
import csv
import os
from tqdm import tqdm  # For progress bar
from datetime import datetime

# Set your identity as required by SEC EDGAR
set_identity("mike@indigo.com")

# Function to get S&P 500 tickers
def get_sp500_tickers():
    # You can use pandas to get S&P 500 tickers from Wikipedia
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500 = tables[0]
    return sp500['Symbol'].tolist()

# Function to extract MD&A text from a company's 10-K filings from 2015 to present
def extract_mda_historical(ticker, start_year=2015, delay=1):
    results = []
    missing_mda = []  # Track filings where MD&A section couldn't be retrieved
    
    try:
        # Create Company object
        company = Company(ticker)
        company_name = company.name
        cik = company.cik
        
        print(f"Processing {ticker} ({company_name})")
        
        # Get all 10-K filings
        filings = company.get_filings(form="10-K")
        
        # Convert filings to DataFrame for easier filtering
        try:
            filings_df = filings.to_pandas()
            
            # Print the columns to understand the data structure
            print(f"  Available columns: {filings_df.columns.tolist()}")
            
            # Filter filings from 2015 onwards based on reportDate
            if 'reportDate' in filings_df.columns:
                # Extract year from reportDate and filter
                filings_df['year'] = pd.to_datetime(filings_df['reportDate']).dt.year
                filtered_df = filings_df[filings_df['year'] >= start_year]
                
                print(f"  Found {len(filtered_df)} 10-K filings from {start_year} onwards")
                
                # Process each filing in the filtered DataFrame
                for _, row in filtered_df.iterrows():
                    try:
                        # Get filing object using the accessionNumber
                        accession_number = row['accessionNumber'] if 'accessionNumber' in row else None
                        filing_url = row['fileUrl'] if 'fileUrl' in row else None
                        
                        # Find the corresponding filing object
                        filing_obj = None
                        for filing in filings:
                            if (accession_number and hasattr(filing, 'accession_number') and 
                                filing.accession_number == accession_number):
                                filing_obj = filing
                                break
                            # Alternative: try by index if the dataframe and filings list maintain the same order
                            if filing_obj is None and filing_url:
                                # Try to match by any other identifier if available
                                pass
                        
                        # If no match found by accession number, try to get the filing by index
                        if filing_obj is None:
                            idx = filtered_df.index.get_loc(row.name)
                            if idx < len(filings):
                                filing_obj = filings[idx]
                        
                        # If we found a filing object, process it
                        if filing_obj:
                            tenk = filing_obj.obj()
                            
                            filing_date = tenk.filing_date
                            
                            # Get Management Discussion section (Item 7)
                            mda_text = tenk.management_discussion
                            
                            # Check if MD&A section was retrieved
                            current_year = row['year'] if 'year' in row else None
                            
                            if not mda_text or mda_text.strip() == "":
                                missing_mda.append({
                                    'Ticker': ticker,
                                    'CIK': cik,
                                    'Year': current_year,
                                    'Filing Date': filing_date
                                })
                                print(f"    WARNING: No MD&A section found for {ticker} (CIK: {cik}) for year {current_year}")
                            else:
                                results.append({
                                    'Ticker': ticker,
                                    'Company Name': company_name,
                                    'CIK': cik,
                                    'Filing Date': filing_date,
                                    'Report Date': row['reportDate'] if 'reportDate' in row else None,
                                    'Year': current_year,
                                    'MD&A Text': mda_text
                                })
                            
                            # Wait to avoid hitting rate limits
                            time.sleep(delay)
                    except Exception as e:
                        print(f"    Error processing filing for {ticker}: {e}")
                        # Track the missing data
                        current_year = row['year'] if 'year' in row else None
                        missing_mda.append({
                            'Ticker': ticker,
                            'CIK': cik,
                            'Year': current_year,
                            'Filing Date': None,
                            'Error': str(e)
                        })
                        continue
            else:
                # If reportDate is not available, process all filings and filter by filing_date
                print("  'reportDate' column not found. Processing all filings and filtering by filing_date.")
                
                for filing in filings:
                    try:
                        tenk = filing.obj()
                        filing_date = tenk.filing_date
                        
                        # Check if filing_date is available and from 2015 or later
                        if filing_date and filing_date.startswith('20'):
                            filing_year = int(filing_date.split('-')[0])
                            if filing_year >= start_year:
                                # Get Management Discussion section (Item 7)
                                mda_text = tenk.management_discussion
                                
                                # Check if MD&A section was retrieved
                                if not mda_text or mda_text.strip() == "":
                                    missing_mda.append({
                                        'Ticker': ticker,
                                        'CIK': cik,
                                        'Year': filing_year,
                                        'Filing Date': filing_date
                                    })
                                    print(f"    WARNING: No MD&A section found for {ticker} (CIK: {cik}) for year {filing_year}")
                                else:
                                    results.append({
                                        'Ticker': ticker,
                                        'Company Name': company_name,
                                        'CIK': cik,
                                        'Filing Date': filing_date,
                                        'Report Date': None,
                                        'Year': filing_year,
                                        'MD&A Text': mda_text
                                    })
                        
                        # Wait to avoid hitting rate limits
                        time.sleep(delay)
                    except Exception as e:
                        print(f"    Error processing filing: {e}")
                        # Can't track year here since we don't know it
                        missing_mda.append({
                            'Ticker': ticker,
                            'CIK': cik,
                            'Year': None,
                            'Filing Date': None,
                            'Error': str(e)
                        })
                        continue
                
        except Exception as e:
            print(f"  Error converting filings to DataFrame: {e}")
            # Fallback: process all filings without DataFrame filtering
            print("  Falling back to processing all filings without DataFrame filtering.")
            
            for filing in filings:
                try:
                    tenk = filing.obj()
                    filing_date = tenk.filing_date
                    
                    # Check if filing_date is available and from 2015 or later
                    if filing_date and filing_date.startswith('20'):
                        filing_year = int(filing_date.split('-')[0])
                        if filing_year >= start_year:
                            # Get Management Discussion section (Item 7)
                            mda_text = tenk.management_discussion
                            
                            # Check if MD&A section was retrieved
                            if not mda_text or mda_text.strip() == "":
                                missing_mda.append({
                                    'Ticker': ticker,
                                    'CIK': cik,
                                    'Year': filing_year,
                                    'Filing Date': filing_date
                                })
                                print(f"    WARNING: No MD&A section found for {ticker} (CIK: {cik}) for year {filing_year}")
                            else:
                                results.append({
                                    'Ticker': ticker,
                                    'Company Name': company_name,
                                    'CIK': cik,
                                    'Filing Date': filing_date,
                                    'Report Date': None,
                                    'Year': filing_year,
                                    'MD&A Text': mda_text
                                })
                    
                    # Wait to avoid hitting rate limits
                    time.sleep(delay)
                except Exception as e:
                    print(f"    Error processing filing: {e}")
                    missing_mda.append({
                        'Ticker': ticker,
                        'CIK': cik,
                        'Year': None,
                        'Filing Date': None,
                        'Error': str(e)
                    })
                    continue
        
        return results, missing_mda
    
    except Exception as e:
        print(f"Error processing company {ticker}: {e}")
        missing_mda.append({
            'Ticker': ticker,
            'CIK': cik if 'cik' in locals() else None,
            'Year': None,
            'Filing Date': None,
            'Error': str(e)
        })
        return [], missing_mda

# Main function
def main():
    # Get S&P 500 tickers
    print("Getting S&P 500 tickers...")
    tickers = get_sp500_tickers()
    
    # Create timestamp for the output files
    # timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = "sp500_management_discussion_2015_to_present.csv"
    missing_mda_file = "sp500_missing_mda_2015_to_present.csv"
    
    # Define start year
    start_year = 2015
    
    # Write CSV header for main output file
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Ticker', 'Company Name', 'CIK', 'Filing Date', 'Report Date', 'Year', 'MD&A Text']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
    # Write CSV header for missing MD&A file
    with open(missing_mda_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Ticker', 'CIK', 'Year', 'Filing Date', 'Error']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
    # Process each ticker
    print(f"Processing {len(tickers)} companies from {start_year} onwards...")
    
    # You can adjust this to process only a subset of companies for testing
    # tickers = tickers[:3]  # Uncomment to test with only 3 companies
    
    all_missing_mda = []
    
    for ticker in tqdm(tickers):
        # Clean ticker symbol (remove dots and special characters)
        clean_ticker = ticker.replace('.', '-')
        
        # Extract historical MD&A
        results, missing_mda = extract_mda_historical(clean_ticker, start_year)
        
        # Append to CSV
        if results:
            with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
                fieldnames = ['Ticker', 'Company Name', 'CIK', 'Filing Date', 'Report Date', 'Year', 'MD&A Text']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                for result in results:
                    writer.writerow(result)
        
        # Append missing MD&A to list
        all_missing_mda.extend(missing_mda)
        
        # Write missing MD&A to CSV after each company to save progress
        if missing_mda:
            with open(missing_mda_file, 'a', newline='', encoding='utf-8') as csvfile:
                fieldnames = ['Ticker', 'CIK', 'Year', 'Filing Date', 'Error']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                for item in missing_mda:
                    writer.writerow(item)
        
        # Add a slight delay between companies to avoid hitting rate limits
        time.sleep(1)
    
    # Print summary
    print(f"Extraction complete.")
    print(f"Successfully extracted MD&A sections: {sum(len(results) for ticker, results, _ in [(t, *extract_mda_historical(t.replace('.', '-'), start_year)) for t in tickers])}")
    print(f"Missing MD&A sections: {len(all_missing_mda)}")
    print(f"Data saved to {output_file}")
    print(f"Missing MD&A information saved to {missing_mda_file}")

if __name__ == "__main__":
    main()

Getting S&P 500 tickers...
Processing 503 companies from 2015 onwards...


  0%|          | 0/503 [00:00<?, ?it/s]

Processing MMM (3M CO)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription']
  Found 10 10-K filings from 2015 onwards
    Error processing filing for MMM: 'NoneType' object is not subscriptable
    Error processing filing for MMM: 'NoneType' object is not subscriptable
    Error processing filing for MMM: 'NoneType' object is not subscriptable
    Error processing filing for MMM: 'NoneType' object is not subscriptable


  0%|          | 1/503 [00:10<1:26:25, 10.33s/it]

Processing AOS (SMITH A O CORP)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription']
  Found 10 10-K filings from 2015 onwards


...

/home/nawa/.local/lib/python3.11/site-packages/edgar/files/htmltools.py:271: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk_df.Item = chunk_df.Item.ffill().infer_objects(copy=False)


/home/nawa/.local/lib/python3.11/site-packages/edgar/files/htmltools.py:271: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk_df.Item = chunk_df.Item.ffill().infer_objects(copy=False)


Processing MOS (MOSAIC CO)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription']
  Found 9 10-K filings from 2015 onwards
Processing MSI (Motorola Solutions, Inc.)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription']
  Found 10 10-K filings from 2015 onwards
Processing MSCI (MSCI Inc.)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription']
  Found 10 10-K filings from 2015 onwards
Processing NDAQ (NASDAQ, INC.)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber'

/home/nawa/.local/lib/python3.11/site-packages/edgar/files/htmltools.py:271: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk_df.Item = chunk_df.Item.ffill().infer_objects(copy=False)


Processing VICI (VICI PROPERTIES INC.)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription']
  Found 8 10-K filings from 2015 onwards
Processing V (VISA INC.)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription']
  Found 10 10-K filings from 2015 onwards
Processing VST (Vistra Corp.)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription']
  Found 8 10-K filings from 2015 onwards
Processing VMC (Vulcan Materials CO)
  Available columns: ['accession_number', 'filing_date', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumbe

# Extract most recent 10-K filings ONLY

In [1]:
# S&P 500 Management Discussion & Analysis Extractor
# This script extracts Item 7 (MD&A) text from the most recent 10-K filings
# of S&P 500 companies and exports it to a CSV file.

import pandas as pd
import time
from edgar import *
import csv
import os
from tqdm import tqdm  # For progress bar

# Set your identity as required by SEC EDGAR
set_identity("mike@indigo.com")

# Function to get S&P 500 tickers
def get_sp500_tickers():
    # You can use pandas to get S&P 500 tickers from Wikipedia
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500 = tables[0]
    return sp500['Symbol'].tolist()

# Function to extract MD&A text from a company's 10-K
def extract_mda(ticker, delay=1):
    try:
        # Create Company object
        company = Company(ticker)
        
        # Get latest 10-K filing
        filing = company.get_filings(form="10-K").latest()
        tenk = filing.obj()
        
        # Extract information
        company_name = company.name
        cik = company.cik
        filing_date = tenk.filing_date
        
        # Get Management Discussion section (Item 7)
        mda_text = tenk.management_discussion
        
        # Wait to avoid hitting rate limits
        time.sleep(delay)
        
        return {
            'Ticker': ticker,
            'Company Name': company_name,
            'CIK': cik,
            'Filing Date': filing_date,
            'MD&A Text': mda_text
        }
    
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return {
            'Ticker': ticker,
            'Company Name': None,
            'CIK': None,
            'Filing Date': None,
            'MD&A Text': str(e)
        }

# Main function
def main():
    # Get S&P 500 tickers
    print("Getting S&P 500 tickers...")
    tickers = get_sp500_tickers()
    
    # Output file
    output_file = "sp500_management_discussion.csv"
    
    # Write CSV header
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Ticker', 'Company Name', 'CIK', 'Filing Date', 'MD&A Text']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
    # Process each ticker
    print(f"Processing {len(tickers)} companies...")
    for ticker in tqdm(tickers):
        # Clean ticker symbol (remove dots and special characters)
        clean_ticker = ticker.replace('.', '-')
        
        # Extract MD&A
        result = extract_mda(clean_ticker)
        
        # Append to CSV
        with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Ticker', 'Company Name', 'CIK', 'Filing Date', 'MD&A Text']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(result)
    
    print(f"Extraction complete. Data saved to {output_file}")

if __name__ == "__main__":
    main()

/home/nawa/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Getting S&P 500 tickers...
Processing 503 companies...


 17%|█▋        | 85/503 [05:03<23:38,  3.39s/it]/home/nawa/.local/lib/python3.11/site-packages/edgar/files/htmltools.py:271: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk_df.Item = chunk_df.Item.ffill().infer_objects(copy=False)
 22%|██▏       | 109/503 [06:20<20:10,  3.07s/it]/home/nawa/.local/lib/python3.11/site-packages/edgar/files/htmltools.py:271: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk_df.Item = chunk_df.Item.ffill().infer_objects(copy=False)
 47%|████▋     | 234/503 [13:52<14:08,  3.16s/it]/home/nawa/.local/lib/python3.11/site-pa

Error processing SW: 'NoneType' object has no attribute 'obj'


100%|██████████| 503/503 [29:05<00:00,  3.47s/it]

Extraction complete. Data saved to sp500_management_discussion.csv
